<a href="https://colab.research.google.com/github/AlexBel1912/Capstone-Epicode/blob/main/DIM_e_Facts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import os

print("Implementazione per ecosistema startup italiane")
# Configurazioni
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
warnings.filterwarnings('ignore')

print("🌐 GOOGLE COLAB - POWER BI DATA PREPARATION")
print("="*60)
print(f"✅ Pandas: {pd.__version__}")
print(f"✅ Numpy: {np.__version__}")
print(f"✅ Runtime Type: {os.environ.get('COLAB_GPU', 'CPU')}")
print("="*60)

Implementazione per ecosistema startup italiane
🌐 GOOGLE COLAB - POWER BI DATA PREPARATION
✅ Pandas: 2.2.2
✅ Numpy: 2.0.2
✅ Runtime Type: 


In [ ]:
# CELLA 2: CARICAMENTO FILE CSV
print("\n📂 CARICAMENTO FILE CSV")
print("="*50)

from google.colab import files
import io # Import io module

def upload_csv_files():
    print("📤 Seleziona i tuoi file CSV...")
    uploaded = files.upload()

    dataframes = {}
    for filename, content in uploaded.items():
        try:
            # Prova diversi separatori
            try:
                df = pd.read_csv(io.BytesIO(content), sep=';', encoding='utf-8')
            except:
                df = pd.read_csv(io.BytesIO(content), sep=',', encoding='utf-8')

            dataframes[filename] = df
            print(f"✅ {filename}: {df.shape[0]:,} righe, {df.shape[1]} colonne")
        except Exception as e:
            print(f"❌ Errore {filename}: {e}")

    return dataframes

# Carica file
print("Carica questi file CSV:")
print("- startup_with_regions.csv")
print("- VEM 2020-2024.csv")

data_files = upload_csv_files()


📂 CARICAMENTO FILE CSV
Carica questi file CSV:
- startup_with_regions.csv
- VEM 2020-2024.csv
📤 Seleziona i tuoi file CSV...


Saving startup_with_regions.csv to startup_with_regions (1).csv
Saving VEM 2020-2024.csv to VEM 2020-2024.csv
✅ startup_with_regions (1).csv: 12,292 righe, 31 colonne
✅ VEM 2020-2024.csv: 1,364 righe, 13 colonne


In [ ]:
# CELLA 3: ASSEGNAZIONE DATAFRAMES

print("\n ASSEGNAZIONE DATAFRAMES")
print("="*50)

# Assegna dataframes basandosi sui nomi file
fact_startup = None
fact_VEM = None

for filename, df in data_files.items():
    print(f" Elaborando: {filename}")

    if 'startup' in filename.lower():
        fact_startup = df
        print(f"Fact_Startup_Registry: {df.shape}")
    elif 'vem 2020-2024' in filename.lower():
        fact_investments = df

# Verifica caricamento
datasets = [
    ("Fact_Startup_Registry", fact_startup),
    ("Fact_VEM_2020-2024", fact_investments),
]

print(f"\nRIEPILOGO CARICAMENTO:")
for name, df in datasets:
    status = "OK" if df is not None else "MANCANTE"
    print(f"   {name}: {status}")


 ASSEGNAZIONE DATAFRAMES
 Elaborando: startup_with_regions (1).csv
Fact_Startup_Registry: (12292, 31)
 Elaborando: VEM 2020-2024.csv

RIEPILOGO CARICAMENTO:
   Fact_Startup_Registry: OK
   Fact_VEM_2020-2024: OK


In [ ]:
# CONTROLLO DATI PRE-CONVERSIONE
print("=== CONTROLLO VEM ORIGINALE ===")
print("Ammontare campione:", fact_investments['Ammontare\nInvestito Totale\n(Euro Mln)'].head())
print("Anno fondazione campione:", fact_investments['Anno di\nFondazione'].head())

=== CONTROLLO VEM ORIGINALE ===
Ammontare campione: 0    0,53
1     1,7
2     0,7
3     1,2
4       3
Name: Ammontare\nInvestito Totale\n(Euro Mln), dtype: object
Anno fondazione campione: 0    2017.0
1    2018.0
2    2017.0
3    2018.0
4    2017.0
Name: Anno di\nFondazione, dtype: float64


In [ ]:
# Pulizia nomi colonne
fact_startup.columns = fact_startup.columns.str.strip()
fact_investments.columns = fact_investments.columns.str.strip()

print("Colonne pulite per tutti i dataset")

Colonne pulite per tutti i dataset


In [ ]:
# Dim_Geography
regioni_startup = set(fact_startup['regione'].dropna().unique())
regioni_vem = set(fact_investments['Regione'].dropna().unique())

all_regions = regioni_startup.union(regioni_vem)

all_regions = sorted([r for r in all_regions if r and r != ''])

dim_geography = pd.DataFrame({
    'RegionKey': [0] + list(range(1, len(all_regions) + 1)),
    'RegionName': ['UNKNOWN'] + all_regions
})

# Unisci tutte le regioni
all_regions = regioni_startup.union(regioni_vem)
all_regions = sorted([r for r in all_regions if r and r != ''])

dim_geography = pd.DataFrame({
    'RegionKey': range(1, len(all_regions) + 1),
    'RegionName': all_regions
})

# Aggiungi macro regioni
macro_regions = {
    'LOMBARDIA': 'Nord',
    'PIEMONTE': 'Nord',
    'VENETO': 'Nord',
    'LIGURIA': 'Nord',
    'EMILIA-ROMAGNA': 'Nord',
    'TRENTINO-ALTO ADIGE': 'Nord',
    'FRIULI-VENEZIA GIULIA': 'Nord',
    "VALLE D'AOSTA": 'Nord',
    'TOSCANA': 'Centro',
    'LAZIO': 'Centro',
    'UMBRIA': 'Centro',
    'MARCHE': 'Centro',
    'ABRUZZO': 'Centro',
    'CAMPANIA': 'Sud',
    'CALABRIA': 'Sud',
    'PUGLIA': 'Sud',
    'BASILICATA': 'Sud',
    'SICILIA': 'Sud',
    'SARDEGNA': 'Sud',
    'MOLISE': 'Sud'
}

dim_geography['MacroRegion'] = dim_geography['RegionName'].map(macro_regions)
dim_geography['MacroRegion'] = dim_geography['MacroRegion'].fillna('UNKNOWN')

print(f"Dim_Geography creata: {len(dim_geography)} records")

# Crea mapping per Region Key
region_mapping = dict(zip(dim_geography['RegionName'], dim_geography['RegionKey']))

print(f"✅ Dim_Geography corretta: {len(dim_geography)} records")

Dim_Geography creata: 21 records
✅ Dim_Geography corretta: 21 records


In [ ]:
import re

def clean_and_standardize_sector(sector_name):
    """
    Pulisce, traduce e standardizza i nomi settori in italiano
    """
    if pd.isna(sector_name) or sector_name == '':
        return 'UNKNOWN'

    # Normalizzazione base
    clean = str(sector_name).strip().lower()

    # Rimuovi caratteri speciali e spazi extra
    clean = re.sub(r'[^\w\s]', ' ', clean)
    clean = re.sub(r'\s+', ' ', clean)
    clean = clean.strip()

    # DIZIONARIO MASTER DI STANDARDIZZAZIONE
    sector_standardization = {
        # ICT & TECH
        'ict': 'Tecnologie dell\'informazione',
        'information technology': 'Tecnologie dell\'informazione',
        'tech': 'Tecnologie dell\'informazione',
        'fintech': 'Tecnologie dell\'informazione',
        'edtech': 'Tecnologie dell\'informazione',
        'proptech': 'Tecnologie dell\'informazione',
        'insurtech': 'Tecnologie dell\'informazione',
        'cleantech': 'Tecnologie dell\'informazione',
        'software': 'Tecnologie dell\'informazione',
        'digital': 'Tecnologie dell\'informazione',
        'artificial intelligence': 'Tecnologie dell\'informazione',
        'blockchain': 'Tecnologie dell\'informazione',
        'cybersecurity': 'Tecnologie dell\'informazione',

        # HEALTHCARE (unificazione)
        'healthcare': 'Sanità e servizi sociali',
        'healthcare and social services': 'Sanità e servizi sociali',
        'health care and social services': 'Sanità e servizi sociali',
        'pharmaceutical and biopharmaceutical industry': 'Sanità e servizi sociali',
        'biotech': 'Sanità e servizi sociali',
        'biotecnologie': 'Sanità e servizi sociali',
        'medtech': 'Sanità e servizi sociali',
        'pharmaceutical': 'Sanità e servizi sociali',
        'life sciences': 'Sanità e servizi sociali',

        # MEDIA & COMMUNICATION (unificazione)
        'media and communication': 'Media e comunicazione',
        'media e comunicazione': 'Media e comunicazione',
        'media': 'Media e comunicazione',
        'communication': 'Media e comunicazione',
        'advertising': 'Media e comunicazione',
        'entertainment': 'Media e comunicazione',
        'gaming': 'Media e comunicazione',

        # INDUSTRIA
        'industria/artigianato': 'Industria e manifatturiero',
        'industria artigianato': 'Industria e manifatturiero',
        'industrial products': 'Industria e manifatturiero',
        'beni e servizi industriali': 'Industria e manifatturiero',
        'manufacturing': 'Industria e manifatturiero',
        'automotive': 'Industria e manifatturiero',
        'aerospace': 'Industria e manifatturiero',
        'aerospaziale': 'Industria e manifatturiero',
        'materials': 'Industria e manifatturiero',
        'nuovi materiali': 'Industria e manifatturiero',
        'nanotecnologie': 'Industria e manifatturiero',
        'mobility': 'Industria e manifatturiero',
        'construction': 'Industria e manifatturiero',
        'edilizia': 'Industria e manifatturiero',

        # SERVIZI
        'servizi': 'Servizi alle imprese',
        'altri servizi': 'Servizi alle imprese',
        'other professional and social services': 'Servizi alle imprese',
        'consulting': 'Servizi alle imprese',
        'logistics': 'Servizi alle imprese',
        'real estate': 'Servizi alle imprese',
        'leisure': 'Servizi alle imprese',
        'tempo libero': 'Servizi alle imprese',

        # FINANCIAL SERVICES
        'servizi finanziari': 'Servizi finanziari',
        'financial services': 'Servizi finanziari',
        'banking': 'Servizi finanziari',
        'insurance': 'Servizi finanziari',
        'asset management': 'Servizi finanziari',

        # CONSUMER & RETAIL
        'consumer goods': 'Beni di consumo',
        'beni di consumo': 'Beni di consumo',
        'retail and wholesale trade': 'Beni di consumo',
        'retail': 'Beni di consumo',
        'fashion': 'Beni di consumo',
        'sports': 'Beni di consumo',

        # FOOD & BEVERAGE
        'food and beverage': 'Alimentare e bevande',
        'food': 'Alimentare e bevande',
        'alimentare': 'Alimentare e bevande',
        'agrifood': 'Alimentare e bevande',

        # AGRICULTURE
        'agricoltura/pesca': 'Agricoltura e pesca',
        'agricoltura pesca': 'Agricoltura e pesca',
        'agricultura': 'Agricoltura e pesca',
        'agricoltura': 'Agricoltura e pesca',
        'agri': 'Agricoltura e pesca',

        # COMMERCE
        'commercio': 'Commercio',
        'trade': 'Commercio',
        'export': 'Commercio',

        # TOURISM
        'turismo': 'Turismo e ospitalità',
        'tourism': 'Turismo e ospitalità',
        'travel': 'Turismo e ospitalità',
        'hospitality': 'Turismo e ospitalità',
        'hotel': 'Turismo e ospitalità',

        # ENERGY & ENVIRONMENT
        'energia e ambiente': 'Energia e ambiente',
        'energy': 'Energia e ambiente',
        'energia e ambients': 'Energia e ambiente',  # typo fix
        'renewable energy': 'Energia e ambiente',
        'environment': 'Energia e ambiente',
        'green': 'Energia e ambiente'
    }

    # Cerca match esatto
    if clean in sector_standardization:
        return sector_standardization[clean]

    # Cerca match parziale (per gestire variazioni)
    for key, value in sector_standardization.items():
        if key in clean or clean in key:
            return value

    # Se non trova match, restituisce versione title case del nome pulito
    return clean.title()

In [ ]:


print(" PULIZIA E STANDARDIZZAZIONE SETTORI")
print("=" * 50)

# Applica pulizia ai 3 dataset
print("📊 Analisi settori pre-pulizia:")
settori_startup_raw = fact_startup['settore'].dropna().unique()
settori_vem_raw = fact_investments['Settore'].dropna().unique()

print(f"- Startup settori: {len(settori_startup_raw)}")
print(f"- VEM settori: {len(settori_vem_raw)}")
print(f"- Totale unique raw: {len(set(list(settori_startup_raw) + list(settori_vem_raw)))}")

# Applica standardizzazione
settori_startup_clean = set()
settori_vem_clean = set()

for settore in settori_startup_raw:
    settori_startup_clean.add(clean_and_standardize_sector(settore))

for settore in settori_vem_raw:
    settori_vem_clean.add(clean_and_standardize_sector(settore))

# Unisci tutti i settori puliti
all_sectors_clean = settori_startup_clean.union(settori_vem_clean)
all_sectors_clean = sorted([s for s in all_sectors_clean if s and s != 'UNKNOWN'])

print("\n📊 Risultati post-pulizia:")
print(f"- Settori unici dopo pulizia: {len(all_sectors_clean)}")
print(f"- Riduzione: {len(set(list(settori_startup_raw) + list(settori_vem_raw) )) - len(all_sectors_clean)} duplicati eliminati")

# Crea Dim_Sector con settori puliti
dim_sector = pd.DataFrame({
    'SectorKey': [0] + list(range(1, len(all_sectors_clean) + 1)),
    'SectorName': ['UNKNOWN'] + all_sectors_clean
})

# Aggiungi categorizzazione macro-settori
def categorize_clean_sector(sector_name):
    """Categorizza settori italiani puliti"""
    if 'Tecnologie dell\'informazione' in sector_name:
        return 'TECNOLOGIA'
    elif 'Sanità' in sector_name:
        return 'SANITÀ'
    elif 'Industria' in sector_name or 'manifatturiero' in sector_name:
        return 'INDUSTRIA'
    elif 'Servizi' in sector_name:
        return 'SERVIZI'
    elif 'Beni di consumo' in sector_name:
        return 'CONSUMER'
    elif 'Alimentare' in sector_name or 'Agricoltura' in sector_name:
        return 'AGROALIMENTARE'
    elif 'Media' in sector_name:
        return 'MEDIA'
    elif 'Energia' in sector_name:
        return 'ENERGIA'
    elif 'Turismo' in sector_name:
        return 'TURISMO'
    elif 'Commercio' in sector_name:
        return 'COMMERCIO'
    else:
        return 'ALTRI'

dim_sector['SectorCategory'] = dim_sector['SectorName'].apply(categorize_clean_sector)

# Crea mapping con settori puliti
sector_mapping = {}
for settore_raw in list(settori_startup_raw) + list(settori_vem_raw):
    settore_clean = clean_and_standardize_sector(settore_raw)
    sector_key = dim_sector[dim_sector['SectorName'] == settore_clean]['SectorKey'].iloc[0] if settore_clean in dim_sector['SectorName'].values else 0
    sector_mapping[settore_raw] = sector_key

print(f"\n✅ Dim_Sector creata: {len(dim_sector)} records")
print("\n📋 Distribuzione categorie:")
category_dist = dim_sector['SectorCategory'].value_counts()
for cat, count in category_dist.items():
    print(f"  {cat}: {count} settori")

print("\n🔍 Sample settori standardizzati:")
for i, row in dim_sector.head(15).iterrows():
    if row['SectorName'] != 'UNKNOWN':
        print(f"  {row['SectorKey']}. {row['SectorName']} → {row['SectorCategory']}")

# Aggiorna funzione di mapping
def get_sector_key(sector_name):
    """Ottiene SectorKey con standardizzazione automatica"""
    if pd.isna(sector_name) or sector_name == '':
        return 0
    return sector_mapping.get(sector_name, 0)

print("\n🎯 SETTORI STANDARDIZZATI E PRONTI PER POWER BI!")



 PULIZIA E STANDARDIZZAZIONE SETTORI
📊 Analisi settori pre-pulizia:
- Startup settori: 5
- VEM settori: 20
- Totale unique raw: 25

📊 Risultati post-pulizia:
- Settori unici dopo pulizia: 12
- Riduzione: 13 duplicati eliminati

✅ Dim_Sector creata: 13 records

📋 Distribuzione categorie:
  AGROALIMENTARE: 2 settori
  SERVIZI: 2 settori
  ALTRI: 1 settori
  CONSUMER: 1 settori
  COMMERCIO: 1 settori
  INDUSTRIA: 1 settori
  ENERGIA: 1 settori
  MEDIA: 1 settori
  SANITÀ: 1 settori
  TECNOLOGIA: 1 settori
  TURISMO: 1 settori

🔍 Sample settori standardizzati:
  1. Agricoltura e pesca → AGROALIMENTARE
  2. Alimentare e bevande → AGROALIMENTARE
  3. Beni di consumo → CONSUMER
  4. Commercio → COMMERCIO
  5. Energia e ambiente → ENERGIA
  6. Industria e manifatturiero → INDUSTRIA
  7. Media e comunicazione → MEDIA
  8. Sanità e servizi sociali → SANITÀ
  9. Servizi alle imprese → SERVIZI
  10. Servizi finanziari → SERVIZI
  11. Tecnologie dell'informazione → TECNOLOGIA
  12. Turismo e ospita

In [ ]:
# Verifica distribuzione categorie
category_dist = dim_sector['SectorCategory'].value_counts()
print("Distribuzione categorie settori:")
for cat, count in category_dist.items():
    print(f"  {cat}: {count} settori")

Distribuzione categorie settori:
  AGROALIMENTARE: 2 settori
  SERVIZI: 2 settori
  ALTRI: 1 settori
  CONSUMER: 1 settori
  COMMERCIO: 1 settori
  INDUSTRIA: 1 settori
  ENERGIA: 1 settori
  MEDIA: 1 settori
  SANITÀ: 1 settori
  TECNOLOGIA: 1 settori
  TURISMO: 1 settori


In [ ]:
def clean_and_standardize_investor(investor_name):
    """
    Pulisce e standardizza i nomi degli investitori
    """
    if pd.isna(investor_name) or investor_name == '':
        return 'UNKNOWN'

    # Normalizzazione base
    clean = str(investor_name).strip()

    # Rimuovi caratteri speciali problematici
    clean = re.sub(r'[^\w\s\.\-&]', ' ', clean)
    clean = re.sub(r'\s+', ' ', clean)
    clean = clean.strip()

    # Standardizzazione forme societarie italiane
    forme_societarie = {
        r'\bs\.r\.l\.?\b': 'Srl',
        r'\bsrl\b': 'Srl',
        r'\bs\.p\.a\.?\b': 'SpA',
        r'\bspa\b': 'SpA',
        r'\bs\.g\.r\.?\b': 'SGR',
        r'\bsgr\b': 'SGR',
        r'\bs\.i\.c\.a\.v\.?\b': 'SICAV',
        r'\bsicav\b': 'SICAV',
        r'\bs\.i\.m\.?\b': 'SIM',
        r'\bsim\b': 'SIM'
    }

    for pattern, replacement in forme_societarie.items():
        clean = re.sub(pattern, replacement, clean, flags=re.IGNORECASE)

    # Standardizzazione termini comuni investment
    investment_terms = {
        r'\bcapital\b': 'Capital',
        r'\bventures?\b': 'Ventures',
        r'\bventure\b': 'Venture',
        r'\bpartners?\b': 'Partners',
        r'\bpartner\b': 'Partners',
        r'\bequity\b': 'Equity',
        r'\bfund\b': 'Fund',
        r'\bfunds\b': 'Fund',
        r'\bmanagement\b': 'Management',
        r'\badvisors?\b': 'Advisors',
        r'\binvestments?\b': 'Investment',
        r'\bholding\b': 'Holding',
        r'\bgroup\b': 'Group',
        r'\binvestors?\b': 'Investor',
        r'\bassociates?\b': 'Associates',
        r'\blimited\b': 'Limited',
        r'\bltd\b': 'Ltd',
        r'\binc\b': 'Inc',
        r'\bcorp\b': 'Corp',
        r'\bcompany\b': 'Company',
        r'\bco\b': 'Co'
    }

    for pattern, replacement in investment_terms.items():
        clean = re.sub(pattern, replacement, clean, flags=re.IGNORECASE)

    # Title Case con gestione delle eccezioni
    words = clean.split()
    standardized_words = []

    # Parole che devono rimanere uppercase
    uppercase_words = {'SGR', 'SRL', 'SPA', 'SICAV', 'SIM', 'LLC', 'LP', 'LTD', 'INC', 'CORP', 'VC', 'PE', 'AI', 'IT', 'ICT', 'R&D', 'M&A'}

    # Parole che devono rimanere lowercase
    lowercase_words = {'and', 'or', 'of', 'for', 'in', 'on', 'at', 'to', 'by', 'with', 'from', 'the', 'a', 'an', 'di', 'e', 'del', 'della', 'dei', 'delle', 'da', 'in', 'con', 'per', 'su'}

    for i, word in enumerate(words):
        word_upper = word.upper()
        word_lower = word.lower()

        if word_upper in uppercase_words:
            standardized_words.append(word_upper)
        elif i > 0 and word_lower in lowercase_words:  # Non apply lowercase to first word
            standardized_words.append(word_lower)
        else:
            # Title case standard
            standardized_words.append(word.title())

    result = ' '.join(standardized_words)

    # Fix per "Cdp" -> "CDP"
    result = re.sub(r'\bCdp\b', 'CDP', result)

    return result



In [ ]:
print("PULIZIA E STANDARDIZZAZIONE INVESTITORI")
print("=" * 50)

# Raccogli investitori da VEM e PEM
investitori_vem_raw = fact_investments['Lead Investor'].dropna().unique()

print("📊 Analisi investitori pre-pulizia:")
print(f"- VEM investitori: {len(investitori_vem_raw)}")


all_investors_raw = list(investitori_vem_raw)
print(f"- Totale unique raw: {len(set(all_investors_raw))}")

# Applica pulizia
investors_cleaned = set()
for investor in all_investors_raw:
    cleaned = clean_and_standardize_investor(investor)
    if cleaned and cleaned != 'UNKNOWN':
        investors_cleaned.add(cleaned)

all_investors_clean = sorted(list(investors_cleaned))

print(f"\n📊 Risultati post-pulizia:")
print(f"- Investitori unici dopo pulizia: {len(all_investors_clean)}")
print(f"- Riduzione: {len(set(all_investors_raw)) - len(all_investors_clean)} duplicati eliminati")
print(f"- Riduzione percentuale: {((len(set(all_investors_raw)) - len(all_investors_clean)) / len(set(all_investors_raw)) * 100):.1f}%")

# Crea Dim_Investor con investitori puliti
dim_investor = pd.DataFrame({
    'InvestorKey': [0] + list(range(1, len(all_investors_clean) + 1)),
    'InvestorName': ['UNKNOWN'] + all_investors_clean
})

# Applica classificazione migliorata
def classify_investor_smart(investor_name):
    """Classificazione investitori con precedenza gerarchica"""
    if pd.isna(investor_name) or investor_name == '' or investor_name == 'UNKNOWN':
        return 'UNKNOWN'

    investor_upper = str(investor_name).upper()

    # PRECEDENZA 1: GOVERNMENT (priorità massima)
    if any(word in investor_upper for word in ['CDP', 'INVITALIA', 'SIMEST', 'MISE', 'REGIONE', 'GOVERNO', 'STATO', 'MIUR', 'EUROPA', 'EUROPEAN']):
        return 'GOVERNMENT'

    # PRECEDENZA 2: CORPORATE VC
    corporate_indicators = ['CORPORATE', 'CVC', 'ACCELERATOR', 'INCUBATOR', 'LABS', 'INNOVATION']
    if any(word in investor_upper for word in corporate_indicators):
        return 'CORPORATE_VC'

    # PRECEDENZA 3: VENTURE CAPITAL
    vc_indicators = ['VENTURE', 'VENTURES', 'VC', 'SEED', 'EARLY']
    if any(word in investor_upper for word in vc_indicators):
        return 'VC'

    # PRECEDENZA 4: PRIVATE EQUITY
    pe_indicators = ['PRIVATE EQUITY', 'PE', 'BUYOUT', 'CAPITAL', 'FUND', 'EQUITY']
    if any(word in investor_upper for word in pe_indicators):
        return 'PE'

    # PRECEDENZA 5: FINANCIAL INSTITUTION
    bank_indicators = ['BANK', 'BANCA', 'BANKING', 'UNICREDIT', 'INTESA', 'MEDIOBANCA', 'CREDIT', 'FINANCIAL']
    if any(word in investor_upper for word in bank_indicators):
        return 'FINANCIAL_INSTITUTION'

    # PRECEDENZA 6: FAMILY OFFICE
    family_indicators = ['FAMILY', 'ANGEL', 'PERSONAL', 'WEALTH', 'OFFICE']
    if any(word in investor_upper for word in family_indicators):
        return 'FAMILY_OFFICE'

    return 'OTHER'

dim_investor['InvestorType'] = dim_investor['InvestorName'].apply(classify_investor_smart)

# Crea mapping da RAW a pulito
investor_mapping = {}
for investor_raw in all_investors_raw:
    investor_clean = clean_and_standardize_investor(investor_raw)
    if investor_clean in dim_investor['InvestorName'].values:
        investor_key = dim_investor[dim_investor['InvestorName'] == investor_clean]['InvestorKey'].iloc[0]
    else:
        investor_key = 0  # UNKNOWN
    investor_mapping[investor_raw] = investor_key

print(f"\n✅ Dim_Investor creata: {len(dim_investor)} records")

# Statistiche per tipologia
investor_type_dist = dim_investor['InvestorType'].value_counts()
print("\n📋 Distribuzione tipologie investitori:")
for tipo, count in investor_type_dist.items():
    print(f"  {tipo}: {count} investitori")

# Sample degli investitori puliti
print("\n🔍 Sample investitori standardizzati:")
for i, row in dim_investor.head(20).iterrows():
    if row['InvestorName'] != 'UNKNOWN':
        print(f"  {row['InvestorKey']}. {row['InvestorName']} → {row['InvestorType']}")

# Aggiorna funzione di mapping
def get_investor_key(investor_name):
    """Ottiene InvestorKey con standardizzazione automatica"""
    if pd.isna(investor_name) or investor_name == '':
        return 0
    return investor_mapping.get(investor_name, 0)

print("\n🎯 INVESTITORI STANDARDIZZATI E PRONTI PER POWER BI!")

# Mostra alcuni esempi di mapping RAW → CLEAN
print("\n📋 Esempi di standardizzazione:")
sample_mapping = list(investor_mapping.items())[:10]
for raw, key in sample_mapping:
    if key > 0:
        clean_name = dim_investor[dim_investor['InvestorKey'] == key]['InvestorName'].iloc[0]
        print(f"  '{raw}' → '{clean_name}'")

PULIZIA E STANDARDIZZAZIONE INVESTITORI
📊 Analisi investitori pre-pulizia:
- VEM investitori: 593
- Totale unique raw: 593

📊 Risultati post-pulizia:
- Investitori unici dopo pulizia: 592
- Riduzione: 1 duplicati eliminati
- Riduzione percentuale: 0.2%

✅ Dim_Investor creata: 593 records

📋 Distribuzione tipologie investitori:
  OTHER: 295 investitori
  VC: 124 investitori
  PE: 85 investitori
  GOVERNMENT: 47 investitori
  FINANCIAL_INSTITUTION: 19 investitori
  CORPORATE_VC: 18 investitori
  FAMILY_OFFICE: 4 investitori
  UNKNOWN: 1 investitori

🔍 Sample investitori standardizzati:
  1. 035 Investimenti Dayone → OTHER
  2. 33N Partech → OTHER
  3. 360 Capital Partners → PE
  4. 360 Capital Partners 14Peaks Capital → PE
  5. 360 Capital Partners Anche Tramite Lv360 → PE
  6. 360 Capital Partners Poli360 → PE
  7. 360 Capital Partners Poli360 Eureka Ventures SGR Eureka Fund I - Technology Transfer → VC
  8. 360 Capital Partners Tech4Planet Polo Tt CDP Ventures Capital SGR → GOVERNMENT


In [ ]:
# Funzione parse_date_to_key migliorata
def parse_date_to_key(date_str):
    """Converte data string in DateKey per relazioni Power BI"""
    if pd.isna(date_str) or date_str == '' or date_str == 'nan':
        return None

    try:
        date_str = str(date_str).strip()

        # Formati data comuni nei dataset italiani
        date_formats = [
            '%d/%m/%Y',      # 25/01/2023
            '%Y-%m-%d',      # 2023-01-25
            '%d-%m-%Y',      # 25-01-2023
            '%d/%m/%Y %H:%M',# 25/01/2023 00:00
            '%Y-%m-%d %H:%M:%S'  # 2023-01-25 00:00:00
        ]

        for fmt in date_formats:
            try:
                date_obj = datetime.strptime(date_str, fmt)
                return int(date_obj.strftime('%Y%m%d'))
            except ValueError:
                continue

        # Se nessun formato funziona, prova parsing automatico
        try:
            date_obj = pd.to_datetime(date_str, dayfirst=True)
            return int(date_obj.strftime('%Y%m%d'))
        except:
            return None

    except Exception:
        return None

# CORREZIONE: Funzioni di mapping che gestiscono valori NULL
def get_region_key(region_name):
    """Ottiene RegionKey da nome regione - gestisce NULL"""
    if pd.isna(region_name) or region_name == '' or region_name == 'nan':
        return 0  # UNKNOWN
    return region_mapping.get(str(region_name).strip(), 0)

def get_sector_key(sector_name):
    """Ottiene SectorKey da nome settore - gestisce NULL"""
    if pd.isna(sector_name) or sector_name == '' or sector_name == 'nan':
        return 0  # UNKNOWN
    return sector_mapping.get(str(sector_name).strip(), 0)

def get_investor_key(investor_name):
    """Ottiene InvestorKey da nome investitore - gestisce NULL"""
    if pd.isna(investor_name) or investor_name == '' or investor_name == 'nan':
        return 0  # UNKNOWN
    return investor_mapping.get(str(investor_name).strip(), 0)

print("✅ Funzioni di utilità corrette - gestione NULL implementata")



✅ Funzioni di utilità corrette - gestione NULL implementata


In [ ]:


# CORREZIONE: Fact_Startup_Registry con DateKey corretto
fact_startup_registry = pd.DataFrame({
    'StartupID': range(1, len(fact_startup) + 1),
    'RegionKey': fact_startup['regione'].apply(get_region_key),
    'SectorKey': fact_startup['settore'].apply(get_sector_key),


    # Campi originali (per reference)
    'Data_Iscrizione_StartUp': fact_startup['data iscrizione alla sezione delle startup'],
    'Data_Inizio_Attivita': fact_startup['data inizio dell\'esercizio effettivo dell\'attività'],
    'Data_Registro_Imprese': fact_startup['data iscrizione al Registro Imprese'],

    # Altri campi
    'Denominazione': fact_startup['denominazione'],
    'nome': fact_startup['nome'],
    'Provincia': fact_startup['pv'],
    'Comune': fact_startup['comune'],
    'Regione': fact_startup['regione'],
    'AtecoCode': fact_startup['ateco 2007'],
    'Settore': fact_startup['settore'],
    'AltoValoreTecnologico': fact_startup['alto valore tecnologico'],
    'Classe_addetti': fact_startup['classe addetti'],
    'Classe_val_produzione': fact_startup['classe val.prod.'],
    'ClasseCapitaleSociale': fact_startup['classe cap.sociale'],
    'SpeseRnD': fact_startup['spese in ricerca e sviluppo'],
    'ForzaLavoroTitoli': fact_startup['forza lavoro con titoli'],
    'PossessoBrevetti': fact_startup['possesso di brevetti'],
    'PrevalenzaFemminile': fact_startup['prevalenza femminile'],
    'PrevalenzaGiovanile': fact_startup['prevalenza giovanile'],
    'PrevalenzaStraniera': fact_startup['prevalenza straniera'],
    'Status': fact_startup['status'],
    'TipoSocieta': fact_startup['tipo_societa'],
    'Year': fact_startup['Year'],
})

print(f"✅ Fact_Startup_Registry corretta: {len(fact_startup_registry)} records")


✅ Fact_Startup_Registry corretta: 12292 records
✅ Fact_Startup_Registry corretta: 12292 records


In [ ]:
#Fact_VEM_Investments

def vem_smart_correction(row):
    """Correzione intelligente basata su analisi pattern"""
    value = row['AmountInvested_Original']
    raw_text = str(row['Amount_Raw']).strip()

    if pd.isna(value) or value == 0:
        return 0.0

    # LOGICA EVIDENCE-BASED:
    # Se valore < 1 OR contiene virgola nel raw → probabilmente corretto
    if value < 1 or ',' in raw_text:
        return value

    # Se valore molto grande → probabilmente moltiplicato
    if value > 50:
        return value / 100

    # Valori medi (1-50) → mantieni come sono
    return value

# APPLICAZIONE UNICA E DEFINITIVA
fact_vem_final = pd.DataFrame({
    'VEMDealID': range(1, len(fact_investments) + 1),
    'RegionKey': fact_investments['Regione'].apply(get_region_key),
    'SectorKey': fact_investments['Settore'].apply(get_sector_key),
    'InvestorKey': fact_investments['Lead Investor'].apply(get_investor_key),

    # Campi business
    'TargetCompany': fact_investments['nome'],
    'LeadInvestor': fact_investments['Lead Investor'],
    'CoInvestors': fact_investments['Co-Investors'],
    'InvestmentStage': fact_investments['Investment\nStage'],
    'DealOrigination': fact_investments['Deal\nOrigination'],
    'Regione': fact_investments['Regione'],
    'Country': fact_investments['Country'],
    'AnnoFondazione': pd.to_numeric(fact_investments['Anno di\nFondazione'], errors='coerce'),
    'Settore': fact_investments['Settore'],
    'DescrizioneAttivita': fact_investments['Descrizione Attività'],
    'AnnoReportVEM': pd.to_numeric(fact_investments['Anno report VEM'], errors='coerce'),

    # DATI HELPER per logica
    'AmountInvested_Original': fact_investments['Ammontare\nInvestito Totale\n(Euro Mln)']
    .astype(str).str.replace(',', '.').replace(['', 'nan'], '0').astype(float),
    'Amount_Raw': fact_investments['Ammontare\nInvestito Totale\n(Euro Mln)']
})

# APPLICA CORREZIONE SMART
fact_vem_final['AmountInvested_EUR_MLN'] = fact_vem_final.apply(vem_smart_correction, axis=1)

# RIMUOVI COLONNE HELPER (keep clean)
fact_vem_final = fact_vem_final.drop(['AmountInvested_Original', 'Amount_Raw'], axis=1)

print(f"✅ VEM FINALE con correzione smart: {len(fact_vem_final)} records")



✅ VEM FINALE con correzione smart: 1364 records


In [ ]:
print("🔍 VERIFICA FINALE PRE-EXPORT:")
print("="*50)

# VEM dovrebbe avere valori come 0.53, 1.7, etc.
print("VEM - Primi 10 valori:")
print(fact_vem_final['AmountInvested_EUR_MLN'].head(10).tolist())


🔍 VERIFICA FINALE PRE-EXPORT:
VEM - Primi 10 valori:
[0.53, 1.7, 0.7, 1.2, 3.0, 0.0, 0.2, 2.3, 1.7, 0.0]


In [ ]:


print("\n--- VALIDAZIONE QUALITÀ DATI ---")

# Startup Registry
startup_completeness = {
    'Regione': fact_startup_registry['RegionKey'].notna().sum() / len(fact_startup_registry),
    'Settore': fact_startup_registry['SectorKey'].notna().sum() / len(fact_startup_registry),
    'Data Iscrizione': fact_startup_registry['Data_Iscrizione_StartUp'].notna().sum() / len(fact_startup_registry)
}

print("Startup Registry - Completezza dati:")
for campo, percentuale in startup_completeness.items():
    print(f"  {campo}: {percentuale:.1%}")

# VEM Investments
vem_completeness = {
    'Regione': fact_vem_final['RegionKey'].notna().sum() / len(fact_vem_final),
    'Settore': fact_vem_final['SectorKey'].notna().sum() / len(fact_vem_final),
    'Investitore': fact_vem_final['InvestorKey'].notna().sum() / len(fact_vem_final),
    'Importo': fact_vem_final['AmountInvested_EUR_MLN'].notna().sum() / len(fact_vem_final)
}

print("\nVEM Investments - Completezza dati:")
for campo, percentuale in vem_completeness.items():
    print(f"  {campo}: {percentuale:.1%}")




--- VALIDAZIONE QUALITÀ DATI ---
Startup Registry - Completezza dati:
  Regione: 100.0%
  Settore: 100.0%
  Data Iscrizione: 100.0%

VEM Investments - Completezza dati:
  Regione: 100.0%
  Settore: 100.0%
  Investitore: 100.0%
  Importo: 100.0%


In [ ]:
print(fact_vem_final['AmountInvested_EUR_MLN'].dtype)
print(fact_vem_final['AmountInvested_EUR_MLN'].head(10))


float64
0    0.53
1    1.70
2    0.70
3    1.20
4    3.00
5    0.00
6    0.20
7    2.30
8    1.70
9    0.00
Name: AmountInvested_EUR_MLN, dtype: float64


In [ ]:


# VERIFICA FINALE PRIMA DELL'EXPORT
print("🔍 VERIFICA FINALE PRE-EXPORT:")
print("=" * 50)

# Verifica VEM (dovrebbe avere valori decimali corretti)
print("VEM - Sample AmountInvested (primi 5):",
      fact_vem_final['AmountInvested_EUR_MLN'].head().tolist())
print("VEM - Max value:", fact_vem_final['AmountInvested_EUR_MLN'].max())
print("VEM - Records:", len(fact_vem_final))

# Verifica dimensioni
print("Dim_Geography:", len(dim_geography))
print("Dim_Sector:", len(dim_sector))
print("Dim_Investor:", len(dim_investor))
print("Fact_Startup_Registry:", len(fact_startup_registry))

print("\n🎯 SE I VALORI SOPRA SONO CORRETTI, PROCEDI CON L'EXPORT!")


🔍 VERIFICA FINALE PRE-EXPORT:
VEM - Sample AmountInvested (primi 5): [0.53, 1.7, 0.7, 1.2, 3.0]
VEM - Max value: 187.76
VEM - Records: 1364
Dim_Geography: 21
Dim_Sector: 13
Dim_Investor: 593
Fact_Startup_Registry: 12292

🎯 SE I VALORI SOPRA SONO CORRETTI, PROCEDI CON L'EXPORT!


In [ ]:
fact_vem_final

,VEMDealID,RegionKey,SectorKey,InvestorKey,TargetCompany,LeadInvestor,CoInvestors,InvestmentStage,DealOrigination,Regione,Country,AnnoFondazione,Settore,DescrizioneAttivita,AnnoReportVEM,AmountInvested_EUR_MLN
0,1,21,11,399,Quindi,open venture,NaN,Seed,private enterprise,VENETO,ITALY,2017.0,ICT,Società che sviluppa una soluzione AI che inte...,2024,0.53
1,2,21,8,553,Patchai,"uv cap, sfem italia",Healthware Ventures,Startup,private enterprise,VENETO,ITALY,2018.0,Healthcare,Startup che ha sviluppato un software per la r...,2021,1.70
2,3,21,8,566,Sibylla Biotech,vertis sgr (vv3tt),NaN,Seed,research spin-off,VENETO,ITALY,2017.0,Biotecnologie,Startup che ha sviluppato una piattaforma volt...,2020,0.70
3,4,21,5,364,Finapp,mito technology (progress tech transfer),"Crédit Agricole Italia, Tech4Planet (Polo Cdp ...",Startup,research spin-off,VENETO,ITALY,2018.0,Energia e ambiente,Startup che ha sviluppato sistemi innovativi d...,2022,1.20
4,5,21,5,401,Rubber Conversion,operatore di venture capital,Liftt,Startup,corporate spin-off,VENETO,ITALY,2017.0,Energia e ambiente,Startup che si occupa di riciclare la gomma da...,2023,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,1360,1,5,522,I-Foria Italia,"tech4planet (polo tt cdp venture capital sgr),...",NaN,Seed,research spin-off,ABRUZZO,ITALY,2023.0,Energia e ambiente,Startup che sviluppa tecnologie per il riciclo...,2024,1.50
1360,1361,1,11,441,Artisan Dna,primo ventures sgr,NaN,Startup,private enterprise,ABRUZZO,ITALY,2014.0,ICT,Piattaforma digitale che fornisce ai rivendito...,2024,0.75
1361,1362,1,3,107,Hydraink,cdp venture capital sgr,"Lsg, Ventiseidieci, Investitori Privati",Startup,corporate spin-off,ABRUZZO,ITALY,2021.0,Beni di consumo,Startup specializzata in prodotti cosmetici,2022,5.00
1362,1363,1,11,386,Hui,nimbus capital,NaN,Startup,private enterprise,ABRUZZO,ITALY,2020.0,ICT,Startup che sviluppa una piattaforma SaaS mult...,2024,25.00


In [ ]:
# Assicurati che AnnoReportVEM sia stringa
fact_vem_final['AnnoReportVEM'] = fact_vem_final['AnnoReportVEM'].astype(str)

# Crea colonna 'Year' con la data 31 dicembre dell'anno indicato
fact_vem_final['Year'] = '31/12/' + fact_vem_final['AnnoReportVEM']
fact_vem_final['Year'] = pd.to_datetime(fact_vem_final['Year'], format='%d/%m/%Y', errors='coerce')

# Visualizza le prime righe
print(fact_vem_final[['AnnoReportVEM', 'Year']].head())

  AnnoReportVEM       Year
0          2024 2024-12-31
1          2021 2021-12-31
2          2020 2020-12-31
3          2022 2022-12-31
4          2023 2023-12-31


In [ ]:
# Converte AnnoFondazione in intero, eliminando i decimali
fact_vem_final['AnnoFondazione'] = pd.to_numeric(fact_vem_final['AnnoFondazione'], errors='coerce').astype('Int64')

# Filtra solo le righe dal 2016 in poi
fact_vem_final = fact_vem_final[fact_vem_final['AnnoFondazione'] >= 2016].copy()

# Visualizza le prime righe
print(fact_vem_final.head())

   VEMDealID  RegionKey  SectorKey  InvestorKey      TargetCompany  \
0          1         21         11          399             Quindi   
1          2         21          8          553            Patchai   
2          3         21          8          566    Sibylla Biotech   
3          4         21          5          364             Finapp   
4          5         21          5          401  Rubber Conversion   

                               LeadInvestor  \
0                              open venture   
1                       uv cap, sfem italia   
2                        vertis sgr (vv3tt)   
3  mito technology (progress tech transfer)   
4              operatore di venture capital   

                                         CoInvestors InvestmentStage  \
0                                                NaN            Seed   
1                                Healthware Ventures         Startup   
2                                                NaN            Seed   
3  Crédi

In [ ]:
fact_vem_final = fact_vem_final.drop(columns=['VEMDealID'])

In [ ]:
fact_vem_final

,RegionKey,SectorKey,InvestorKey,TargetCompany,LeadInvestor,CoInvestors,InvestmentStage,DealOrigination,Regione,Country,AnnoFondazione,Settore,DescrizioneAttivita,AnnoReportVEM,AmountInvested_EUR_MLN,Year
0,21,11,399,Quindi,open venture,NaN,Seed,private enterprise,VENETO,ITALY,2017,ICT,Società che sviluppa una soluzione AI che inte...,2024,0.53,2024-12-31
1,21,8,553,Patchai,"uv cap, sfem italia",Healthware Ventures,Startup,private enterprise,VENETO,ITALY,2018,Healthcare,Startup che ha sviluppato un software per la r...,2021,1.70,2021-12-31
2,21,8,566,Sibylla Biotech,vertis sgr (vv3tt),NaN,Seed,research spin-off,VENETO,ITALY,2017,Biotecnologie,Startup che ha sviluppato una piattaforma volt...,2020,0.70,2020-12-31
3,21,5,364,Finapp,mito technology (progress tech transfer),"Crédit Agricole Italia, Tech4Planet (Polo Cdp ...",Startup,research spin-off,VENETO,ITALY,2018,Energia e ambiente,Startup che ha sviluppato sistemi innovativi d...,2022,1.20,2022-12-31
4,21,5,401,Rubber Conversion,operatore di venture capital,Liftt,Startup,corporate spin-off,VENETO,ITALY,2017,Energia e ambiente,Startup che si occupa di riciclare la gomma da...,2023,3.00,2023-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,1,11,470,Vbite,scientifica vc,NaN,Seed,private enterprise,ABRUZZO,ITALY,2022,ICT,Startup che ha creato uno strumento vibrante p...,2022,0.15,2022-12-31
1359,1,5,522,I-Foria Italia,"tech4planet (polo tt cdp venture capital sgr),...",NaN,Seed,research spin-off,ABRUZZO,ITALY,2023,Energia e ambiente,Startup che sviluppa tecnologie per il riciclo...,2024,1.50,2024-12-31
1361,1,3,107,Hydraink,cdp venture capital sgr,"Lsg, Ventiseidieci, Investitori Privati",Startup,corporate spin-off,ABRUZZO,ITALY,2021,Beni di consumo,Startup specializzata in prodotti cosmetici,2022,5.00,2022-12-31
1362,1,11,386,Hui,nimbus capital,NaN,Startup,private enterprise,ABRUZZO,ITALY,2020,ICT,Startup che sviluppa una piattaforma SaaS mult...,2024,25.00,2024-12-31


In [ ]:
fact_vem_final = fact_vem_final.drop(columns=['LeadInvestor'])

In [ ]:
fact_vem_final = fact_vem_final.rename(columns={'AmountInvested_EUR_MLN': 'Amount_Invested'})
print(" Colonna rinominata in 'Amount_Invested'")

 Colonna rinominata in 'Amount_Invested'


In [ ]:
fact_vem_final['Amount_Invested'] = fact_vem_final['Amount_Invested'].astype(str).str.replace(',', '.', regex=False)
fact_vem_final['Amount_Invested'] = pd.to_numeric(fact_vem_final['Amount_Invested'], errors='coerce') * 1_000_000

In [ ]:
fact_vem_final

,RegionKey,SectorKey,InvestorKey,TargetCompany,CoInvestors,InvestmentStage,DealOrigination,Regione,Country,AnnoFondazione,Settore,DescrizioneAttivita,AnnoReportVEM,Amount_Invested,Year
0,21,11,399,Quindi,NaN,Seed,private enterprise,VENETO,ITALY,2017,ICT,Società che sviluppa una soluzione AI che inte...,2024,530000.0,2024-12-31
1,21,8,553,Patchai,Healthware Ventures,Startup,private enterprise,VENETO,ITALY,2018,Healthcare,Startup che ha sviluppato un software per la r...,2021,1700000.0,2021-12-31
2,21,8,566,Sibylla Biotech,NaN,Seed,research spin-off,VENETO,ITALY,2017,Biotecnologie,Startup che ha sviluppato una piattaforma volt...,2020,700000.0,2020-12-31
3,21,5,364,Finapp,"Crédit Agricole Italia, Tech4Planet (Polo Cdp ...",Startup,research spin-off,VENETO,ITALY,2018,Energia e ambiente,Startup che ha sviluppato sistemi innovativi d...,2022,1200000.0,2022-12-31
4,21,5,401,Rubber Conversion,Liftt,Startup,corporate spin-off,VENETO,ITALY,2017,Energia e ambiente,Startup che si occupa di riciclare la gomma da...,2023,3000000.0,2023-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,1,11,470,Vbite,NaN,Seed,private enterprise,ABRUZZO,ITALY,2022,ICT,Startup che ha creato uno strumento vibrante p...,2022,150000.0,2022-12-31
1359,1,5,522,I-Foria Italia,NaN,Seed,research spin-off,ABRUZZO,ITALY,2023,Energia e ambiente,Startup che sviluppa tecnologie per il riciclo...,2024,1500000.0,2024-12-31
1361,1,3,107,Hydraink,"Lsg, Ventiseidieci, Investitori Privati",Startup,corporate spin-off,ABRUZZO,ITALY,2021,Beni di consumo,Startup specializzata in prodotti cosmetici,2022,5000000.0,2022-12-31
1362,1,11,386,Hui,NaN,Startup,private enterprise,ABRUZZO,ITALY,2020,ICT,Startup che sviluppa una piattaforma SaaS mult...,2024,25000000.0,2024-12-31


In [ ]:
fact_startup_registry

,StartupID,RegionKey,SectorKey,Data_Iscrizione_StartUp,Data_Inizio_Attivita,Data_Registro_Imprese,Denominazione,nome,Provincia,Comune,Regione,AtecoCode,Settore,AltoValoreTecnologico,Classe_addetti,Classe_val_produzione,ClasseCapitaleSociale,SpeseRnD,ForzaLavoroTitoli,PossessoBrevetti,PrevalenzaFemminile,PrevalenzaGiovanile,PrevalenzaStraniera,Status,TipoSocieta,Year
0,1,10,9,25/01/2023,12/01/2023,25/01/2023,plw 1 societa' agricola s.r.l.,Plw 1,BG,BERGAMO,LOMBARDIA,72102,SERVIZI,SI,no data,no data,30.0,SI,SI,no data,no data,NO,NO,attiva,societa agricola,2023
1,2,4,9,11/07/2017,12/04/2017,31/05/2017,gate 42 s.r.l. ...,Gate 42,NaN,TORRE DEL GRECO,CAMPANIA,621000,SERVIZI,NO,A,B,60.0,SI,no data,no data,NO,NO,NO,attiva,startup costituita a norma dell art 4 comma 10...,2017
2,3,8,9,23/12/2020,18/12/2020,02/02/2024,block-data 52 - societa' a responsabilita' lim...,Block-Data 52,RM,ROMA,LAZIO,621000,SERVIZI,NO,no data,B,40.0,no data,no data,SI,NO,NO,NO,attiva,startup,2020
3,4,8,0,10/02/2022,10/01/2021,31/03/2025,e-solutions 78 - societa' a responsabilita' li...,E-Solutions 78,RM,ROMA,LAZIO,NC,NaN,NO,no data,B,40.0,no data,SI,no data,NO,NO,NO,attiva,startup,2022
4,5,3,9,22/06/2023,08/06/2023,22/06/2023,before 1783 s.b.s.r.l.,Before 1783,RC,TERRANOVA SAPPO MINULIO,CALABRIA,621000,SERVIZI,NO,A,no data,30.0,SI,no data,no data,Forte,NO,NO,attiva,societa benefit,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12287,12288,8,9,19/07/2023,23/11/2022,05/08/2022,brandsbrokers s.r.l.,Brandsbrokers,RM,ROMA,LAZIO,621000,SERVIZI,NO,A,C,30.0,no data,SI,no data,NO,NO,NO,attiva,societa benefit,2023
12288,12289,18,9,13/01/2022,31/12/2021,14/12/2021,"404 societa' cooperativa sociale, in breve 40...",404,BZ,BOLZANO,TRENTINO-ALTO ADIGE,621000,SERVIZI,NO,B,A,NaN,SI,no data,no data,NO,NO,NO,attiva,societa cooperativa sociale,2022
12289,12290,11,1,12/01/2021,24/12/2020,12/01/2021,vallesina bio s.b. agricola srl,Vallesina,AN,MONSANO,MARCHE,011100,AGRICOLTURA/PESCA,NO,no data,A,30.0,no data,SI,no data,NO,NO,NO,attiva,societa benefit agricola,2021
12290,12291,10,9,24/03/2021,12/03/2021,24/03/2021,zero impack s.r.l. societa' benefit o in forma...,Zero,MI,MILANO,LOMBARDIA,621000,SERVIZI,NO,A,B,40.0,SI,no data,no data,Maggioritaria,NO,NO,attiva,societa benefit,2021


In [ ]:
# assicurarmi che ClasseCapitaleSociale sia numerica
fact_startup_registry['ClasseCapitaleSociale'] = pd.to_numeric(
    fact_startup_registry['ClasseCapitaleSociale'], errors='coerce'
)

# Dividere per 10 e convertilo in intero
fact_startup_registry['ClasseCapitaleSociale'] = (
    fact_startup_registry['ClasseCapitaleSociale'] / 10
).round().astype('Int64')  # Usa Int64 per supportare anche i NaN

# Mostra le prime righe per verifica
print(fact_startup_registry[['ClasseCapitaleSociale']].head())

   ClasseCapitaleSociale
0                      3
1                      6
2                      4
3                      4
4                      3


In [ ]:
print("\n📂 CARICAMENTO FILE EXCEL")
print("="*50)

from google.colab import files
import io
import pandas as pd

def upload_excel_file():
    print("📤 Seleziona il tuo file .xlsx...")
    uploaded = files.upload()

    all_sheets = {}
    for filename, content in uploaded.items():
        try:
            # Carica tutti i fogli del file Excel come dizionario
            xls = pd.read_excel(io.BytesIO(content), sheet_name=None)

            # Stampa il riepilogo dei fogli
            print(f"\n✅ {filename} contiene i seguenti fogli:")
            for name in xls.keys():
                print(f" - {name} ({xls[name].shape[0]} righe, {xls[name].shape[1]} colonne)")

            all_sheets[filename] = xls
        except Exception as e:
            print(f"❌ Errore nel file {filename}: {e}")

    return all_sheets

# Avvia upload e stampa fogli
excel_files = upload_excel_file()



📂 CARICAMENTO FILE EXCEL
📤 Seleziona il tuo file .xlsx...


Saving Prevalenza.xlsx to Prevalenza.xlsx

✅ Prevalenza.xlsx contiene i seguenti fogli:
 - Classe di produzione (8 righe, 2 colonne)
 - Classe di Addetti (16 righe, 3 colonne)
 - Classe di Capitale (11 righe, 2 colonne)
 - Prevalenza (4 righe, 2 colonne)
 - Riepilogo (21 righe, 2 colonne)


In [ ]:
# ASSEGNAZIONE DATAFRAMES

print("\n ASSEGNAZIONE DATAFRAMES")
print("="*50)

# Assegna dataframes basandosi sui nomi dei fogli Excel
produzione = None
addetti = None
capitale = None
prevalenza = None
riepilogo = None # Aggiungi questa variabile per il foglio Riepilogo

for filename, sheets in excel_files.items():
    print(f" Elaborando file: {filename}")
    for sheet_name, df in sheets.items():
        print(f"   Elaborando foglio: {sheet_name}")
        if 'classe di produzione' in sheet_name.lower():
            produzione = df
            print(f"     DataFrame 'produzione': {df.shape}")
        elif 'classe di addetti' in sheet_name.lower():
            addetti = df
            print(f"     DataFrame 'addetti': {df.shape}")
        elif 'classe di capitale' in sheet_name.lower():
            capitale = df
            print(f"     DataFrame 'capitale': {df.shape}")
        elif 'prevalenza' in sheet_name.lower():
            prevalenza = df
            print(f"     DataFrame 'prevalenza': {df.shape}")
        elif 'riepilogo' in sheet_name.lower(): # Aggiungi gestione per il foglio Riepilogo
            riepilogo = df
            print(f"     DataFrame 'riepilogo': {df.shape}")


# Verifica caricamento
datasets = [
    ("produzione", produzione),
    ("addetti", addetti),
    ("capitale", capitale),
    ("prevalenza", prevalenza),
    ("riepilogo", riepilogo) # Aggiungi alla verifica
]

print(f"\nRIEPILOGO CARICAMENTO:")
for name, df in datasets:
    status = "OK" if df is not None else "MANCANTE"
    shape_info = df.shape if df is not None else "N/A"
    print(f"   {name}: {status} {shape_info}")


 ASSEGNAZIONE DATAFRAMES
 Elaborando file: Prevalenza.xlsx
   Elaborando foglio: Classe di produzione
     DataFrame 'produzione': (8, 2)
   Elaborando foglio: Classe di Addetti
     DataFrame 'addetti': (16, 3)
   Elaborando foglio: Classe di Capitale
     DataFrame 'capitale': (11, 2)
   Elaborando foglio: Prevalenza
     DataFrame 'prevalenza': (4, 2)
   Elaborando foglio: Riepilogo
     DataFrame 'riepilogo': (21, 2)

RIEPILOGO CARICAMENTO:
   produzione: OK (8, 2)
   addetti: OK (16, 3)
   capitale: OK (11, 2)
   prevalenza: OK (4, 2)
   riepilogo: OK (21, 2)


In [ ]:
produzione

,valore di produzione,Classe_val_produzione
0,0-100.000 euro,A
1,100.001 - 500.000 euro,B
2,500.001 - 1.000.000 euro,C
3,1.000.001 - 2.000.000 euro,D
4,2.000.001 - 5.000.000 euro,E
5,5.000.001 - 10.000.000 euro,F
6,10.000.001 - 50.000.000 euro,G
7,Più di 50.000.000 euro,H


In [ ]:
# che la colonna per merge abbia lo stesso tipo e valori coerenti
fact_startup_registry['Classe_val_produzione'] = fact_startup_registry['Classe_val_produzione'].astype(str).str.strip().str.upper()
produzione['Classe_val_produzione'] = produzione['Classe_val_produzione'].astype(str).str.strip().str.upper()

# Merge sui valori comuni
fact_startup_merged = pd.merge(
    fact_startup_registry,
    produzione[['Classe_val_produzione', 'valore di produzione']],
    on='Classe_val_produzione',
    how='left'
)

# Elimina la colonna Classe_val_produzione
fact_startup_merged = fact_startup_merged.drop(columns=['Classe_val_produzione'])

In [ ]:
addetti

,addetti,Classe_addetti,Unnamed: 2
0,0-4,A,NaN
1,5-9,B,NaN
2,10-19,C,NaN
3,20-49,D,NaN
4,50-249,E,NaN
5,Almeno 250,F,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [ ]:
# Seleziona solo le righe 0-5 e le due colonne utili
addetti_clean = addetti.loc[0:5, ['addetti', 'Classe_addetti']].copy()

# Mostra il risultato pulito
print("Tabella 'addetti' pulita:")
display(addetti_clean)

Tabella 'addetti' pulita:


,addetti,Classe_addetti
0,0-4,A
1,5-9,B
2,10-19,C
3,20-49,D
4,50-249,E
5,Almeno 250,F


In [ ]:
# che la colonna per merge abbia lo stesso tipo e valori coerenti
fact_startup_merged['Classe_addetti'] = fact_startup_merged['Classe_addetti'].astype(str).str.strip().str.upper()
addetti_clean['Classe_addetti'] = addetti_clean['Classe_addetti'].astype(str).str.strip().str.upper()

# Merge sui valori comuni
fact_startup_merged = pd.merge(
    fact_startup_merged, # Merge with the already merged dataframe
    addetti_clean[['Classe_addetti', 'addetti']],
    on='Classe_addetti',
    how='left'
)

# Elimina la colonna Classe_addetti after the merge
fact_startup_merged = fact_startup_merged.drop(columns=['Classe_addetti'])

In [ ]:
fact_startup_merged

,StartupID,RegionKey,SectorKey,Data_Iscrizione_StartUp,Data_Inizio_Attivita,Data_Registro_Imprese,Denominazione,nome,Provincia,Comune,Regione,AtecoCode,Settore,AltoValoreTecnologico,ClasseCapitaleSociale,SpeseRnD,ForzaLavoroTitoli,PossessoBrevetti,PrevalenzaFemminile,PrevalenzaGiovanile,PrevalenzaStraniera,Status,TipoSocieta,Year,valore di produzione,addetti
0,1,10,9,25/01/2023,12/01/2023,25/01/2023,plw 1 societa' agricola s.r.l.,Plw 1,BG,BERGAMO,LOMBARDIA,72102,SERVIZI,SI,3,SI,SI,no data,no data,NO,NO,attiva,societa agricola,2023,NaN,NaN
1,2,4,9,11/07/2017,12/04/2017,31/05/2017,gate 42 s.r.l. ...,Gate 42,NaN,TORRE DEL GRECO,CAMPANIA,621000,SERVIZI,NO,6,SI,no data,no data,NO,NO,NO,attiva,startup costituita a norma dell art 4 comma 10...,2017,100.001 - 500.000 euro,0-4
2,3,8,9,23/12/2020,18/12/2020,02/02/2024,block-data 52 - societa' a responsabilita' lim...,Block-Data 52,RM,ROMA,LAZIO,621000,SERVIZI,NO,4,no data,no data,SI,NO,NO,NO,attiva,startup,2020,100.001 - 500.000 euro,NaN
3,4,8,0,10/02/2022,10/01/2021,31/03/2025,e-solutions 78 - societa' a responsabilita' li...,E-Solutions 78,RM,ROMA,LAZIO,NC,NaN,NO,4,no data,SI,no data,NO,NO,NO,attiva,startup,2022,100.001 - 500.000 euro,NaN
4,5,3,9,22/06/2023,08/06/2023,22/06/2023,before 1783 s.b.s.r.l.,Before 1783,RC,TERRANOVA SAPPO MINULIO,CALABRIA,621000,SERVIZI,NO,3,SI,no data,no data,Forte,NO,NO,attiva,societa benefit,2023,NaN,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12287,12288,8,9,19/07/2023,23/11/2022,05/08/2022,brandsbrokers s.r.l.,Brandsbrokers,RM,ROMA,LAZIO,621000,SERVIZI,NO,3,no data,SI,no data,NO,NO,NO,attiva,societa benefit,2023,500.001 - 1.000.000 euro,0-4
12288,12289,18,9,13/01/2022,31/12/2021,14/12/2021,"404 societa' cooperativa sociale, in breve 40...",404,BZ,BOLZANO,TRENTINO-ALTO ADIGE,621000,SERVIZI,NO,<NA>,SI,no data,no data,NO,NO,NO,attiva,societa cooperativa sociale,2022,0-100.000 euro,5-9
12289,12290,11,1,12/01/2021,24/12/2020,12/01/2021,vallesina bio s.b. agricola srl,Vallesina,AN,MONSANO,MARCHE,011100,AGRICOLTURA/PESCA,NO,3,no data,SI,no data,NO,NO,NO,attiva,societa benefit agricola,2021,0-100.000 euro,NaN
12290,12291,10,9,24/03/2021,12/03/2021,24/03/2021,zero impack s.r.l. societa' benefit o in forma...,Zero,MI,MILANO,LOMBARDIA,621000,SERVIZI,NO,4,SI,no data,no data,Maggioritaria,NO,NO,attiva,societa benefit,2021,100.001 - 500.000 euro,0-4


In [ ]:
capitale

,valore di capitale,CLASSE DI CAPITALE
0,1 euro,1
1,Da 1 fino a 5mila euro,2
2,Da 5 a 10mila euro,3
3,Da 10 a 50mila euro,4
4,Da 50 a 100mila euro,5
5,Da 100 a 250mila euro,6
6,Da 250 a 500mila euro,7
7,Da 500 a 1mln euro,8
8,"Da 1 a 2,5mln euro",9
9,"Da 2,5 a 5mln euro",10


In [ ]:
# Rinomina la colonna
capitale = capitale.rename(columns={'CLASSE DI CAPITALE': 'ClasseCapitaleSociale'})

# Mostra le prime righe per conferma
print("✅ Colonna rinominata:")
display(capitale.head())

✅ Colonna rinominata:


,valore di capitale,ClasseCapitaleSociale
0,1 euro,1
1,Da 1 fino a 5mila euro,2
2,Da 5 a 10mila euro,3
3,Da 10 a 50mila euro,4
4,Da 50 a 100mila euro,5


In [ ]:
# che la colonna per merge abbia lo stesso tipo e valori coerenti
fact_startup_merged['ClasseCapitaleSociale'] = fact_startup_merged['ClasseCapitaleSociale'].astype(str).str.strip().str.upper()
fact_startup_merged['ClasseCapitaleSociale'] = pd.to_numeric(fact_startup_merged['ClasseCapitaleSociale'], errors='coerce').astype('Int64') # Convert to Int64 after cleaning
capitale['ClasseCapitaleSociale'] = capitale['ClasseCapitaleSociale'].astype(str).str.strip().astype('Int64') # Convert to Int64 to match fact_startup_merged

# Merge sui valori comuni
fact_startup_merged = pd.merge(
    fact_startup_merged, # Merge with the already merged dataframe
    capitale[['valore di capitale', 'ClasseCapitaleSociale']],
    on='ClasseCapitaleSociale',
    how='left'
)

# Elimina la colonna ClasseCapitaleSociale after the merge
fact_startup_merged = fact_startup_merged.drop(columns=['ClasseCapitaleSociale'])

KeyError: 'ClasseCapitaleSociale'

In [ ]:
fact_startup_merged

,StartupID,RegionKey,SectorKey,Data_Iscrizione_StartUp,Data_Inizio_Attivita,Data_Registro_Imprese,Denominazione,nome,Provincia,Comune,Regione,AtecoCode,Settore,AltoValoreTecnologico,SpeseRnD,ForzaLavoroTitoli,PossessoBrevetti,PrevalenzaFemminile,PrevalenzaGiovanile,PrevalenzaStraniera,Status,TipoSocieta,Year,valore di produzione,addetti,valore di capitale
0,1,10,9,25/01/2023,12/01/2023,25/01/2023,plw 1 societa' agricola s.r.l.,Plw 1,BG,BERGAMO,LOMBARDIA,72102,SERVIZI,SI,SI,SI,no data,no data,NO,NO,attiva,societa agricola,2023,NaN,NaN,Da 5 a 10mila euro
1,2,4,9,11/07/2017,12/04/2017,31/05/2017,gate 42 s.r.l. ...,Gate 42,NaN,TORRE DEL GRECO,CAMPANIA,621000,SERVIZI,NO,SI,no data,no data,NO,NO,NO,attiva,startup costituita a norma dell art 4 comma 10...,2017,100.001 - 500.000 euro,0-4,Da 100 a 250mila euro
2,3,8,9,23/12/2020,18/12/2020,02/02/2024,block-data 52 - societa' a responsabilita' lim...,Block-Data 52,RM,ROMA,LAZIO,621000,SERVIZI,NO,no data,no data,SI,NO,NO,NO,attiva,startup,2020,100.001 - 500.000 euro,NaN,Da 10 a 50mila euro
3,4,8,0,10/02/2022,10/01/2021,31/03/2025,e-solutions 78 - societa' a responsabilita' li...,E-Solutions 78,RM,ROMA,LAZIO,NC,NaN,NO,no data,SI,no data,NO,NO,NO,attiva,startup,2022,100.001 - 500.000 euro,NaN,Da 10 a 50mila euro
4,5,3,9,22/06/2023,08/06/2023,22/06/2023,before 1783 s.b.s.r.l.,Before 1783,RC,TERRANOVA SAPPO MINULIO,CALABRIA,621000,SERVIZI,NO,SI,no data,no data,Forte,NO,NO,attiva,societa benefit,2023,NaN,0-4,Da 5 a 10mila euro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12287,12288,8,9,19/07/2023,23/11/2022,05/08/2022,brandsbrokers s.r.l.,Brandsbrokers,RM,ROMA,LAZIO,621000,SERVIZI,NO,no data,SI,no data,NO,NO,NO,attiva,societa benefit,2023,500.001 - 1.000.000 euro,0-4,Da 5 a 10mila euro
12288,12289,18,9,13/01/2022,31/12/2021,14/12/2021,"404 societa' cooperativa sociale, in breve 40...",404,BZ,BOLZANO,TRENTINO-ALTO ADIGE,621000,SERVIZI,NO,SI,no data,no data,NO,NO,NO,attiva,societa cooperativa sociale,2022,0-100.000 euro,5-9,NaN
12289,12290,11,1,12/01/2021,24/12/2020,12/01/2021,vallesina bio s.b. agricola srl,Vallesina,AN,MONSANO,MARCHE,011100,AGRICOLTURA/PESCA,NO,no data,SI,no data,NO,NO,NO,attiva,societa benefit agricola,2021,0-100.000 euro,NaN,Da 5 a 10mila euro
12290,12291,10,9,24/03/2021,12/03/2021,24/03/2021,zero impack s.r.l. societa' benefit o in forma...,Zero,MI,MILANO,LOMBARDIA,621000,SERVIZI,NO,SI,no data,no data,Maggioritaria,NO,NO,attiva,societa benefit,2021,100.001 - 500.000 euro,0-4,Da 10 a 50mila euro


In [ ]:
fact_startup_registry = fact_startup_merged.drop(columns=['StartupID'])

In [ ]:
# Mostra le prime righe
print("✅ Amount_Invested formattato:")
# Format 'Amount_Invested' column for display
display(fact_vem_final[['Amount_Invested']].head().style.format({'Amount_Invested': '{:,.0f}'}))

✅ Amount_Invested formattato:


,Amount_Invested
0,"530,000"
1,"1,700,000"
2,"700,000"
3,"1,200,000"
4,"3,000,000"


In [ ]:
 #EXPORT FINALE

print("\n EXPORT FINALE PER POWER BI")
print("="*60)

# Tabelle da esportare
tables_to_export = {}

if fact_startup_registry is not None:
    tables_to_export['Fact_Startup_Registry'] = fact_startup_registry
if fact_vem_final is not None:
    tables_to_export['Fact_VEM_Investments'] = fact_vem_final
if dim_geography is not None:
    tables_to_export['Dim_Geography'] = dim_geography
if dim_sector is not None:
    tables_to_export['Dim_Sector'] = dim_sector
if dim_investor is not None:
    tables_to_export['Dim_Investor'] = dim_investor

# Export e download
exported_files = []
total_rows = 0

for table_name, df in tables_to_export.items():
    try:
        filename = f"{table_name}.csv"

        # Esporta CSV
        df.to_csv(filename, index=False, encoding='utf-8', sep=',')

        # Statistiche
        rows = df.shape[0]
        cols = df.shape[1]
        total_rows += rows

        print(f" {table_name}:")
        print(f"   • Righe: {rows:,}")
        print(f"   • Colonne: {cols}")
        print(f"   • File: {filename}")

        # Download automatico
        files.download(filename)
        exported_files.append(filename)

        print(f"   ✅ Download avviato!")

    except Exception as e:
        print(f"   ❌ Errore {table_name}: {e}")

print(f"\n EXPORT COMPLETATO!")
print(f" File esportati: {len(exported_files)}")
print(f"💾Righe totali: {total_rows:,}")

print(f"\n NEXT STEPS POWER BI:")
print("   1.  Importa tutti i CSV in Power BI")
print("   2.  Crea relazioni star schema indicate sopra")
print("   3.  Crea DIM_Date con DAX")
print("   4.  Costruisci dashboard con analisi unificate!")
print("   5.  Analizza Registry vs VEM vs Unified data!")


 EXPORT FINALE PER POWER BI
 Fact_Startup_Registry:
   • Righe: 12,292
   • Colonne: 25
   • File: Fact_Startup_Registry.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

   ✅ Download avviato!
 Fact_VEM_Investments:
   • Righe: 1,041
   • Colonne: 15
   • File: Fact_VEM_Investments.csv


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

   ✅ Download avviato!
 Dim_Investor:
   • Righe: 593
   • Colonne: 3
   • File: Dim_Investor.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

   ✅ Download avviato!

 EXPORT COMPLETATO!
 File esportati: 5
💾Righe totali: 13,960

 NEXT STEPS POWER BI:
   1.  Importa tutti i CSV in Power BI
   2.  Crea relazioni star schema indicate sopra
   3.  Crea DIM_Date con DAX
   4.  Costruisci dashboard con analisi unificate!
   5.  Analizza Registry vs VEM vs Unified data!


In [ ]:
print("\n📂 CARICAMENTO FILE CSV")
print("="*50)

from google.colab import files
import io # Import io module

def upload_csv_files():
    print("📤 Seleziona il tuo file CSV...")
    uploaded = files.upload()

    dataframes = {}
    for filename, content in uploaded.items():
        try:
            # Prova diversi separatori
            try:
                df = pd.read_csv(io.BytesIO(content), sep=';', encoding='utf-8')
            except:
                df = pd.read_csv(io.BytesIO(content), sep=',', encoding='utf-8')

            dataframes[filename] = df
            print(f"✅ {filename}: {df.shape[0]:,} righe, {df.shape[1]} colonne")
        except Exception as e:
            print(f"❌ Errore {filename}: {e}")

    return dataframes

# Carica file
print("Carica il file CSV:")

data_files = upload_csv_files()


📂 CARICAMENTO FILE CSV
Carica il file CSV:
📤 Seleziona il tuo file CSV...


Saving georef-italy-comune.csv to georef-italy-comune.csv
✅ georef-italy-comune.csv: 7,904 righe, 17 colonne


In [ ]:
try:
    # Try reading with comma delimiter first
    df = pd.read_csv("georef-italy-comune.csv", sep=',')
except Exception as e:
    print(f"Error reading with comma delimiter: {e}")
    try:
        # If comma fails, try with python engine (more flexible)
        df = pd.read_csv("georef-italy-comune.csv", engine='python')
    except Exception as e_python:
        print(f"Error reading with python engine: {e_python}")
        # Optionally, you could try other delimiters like ';' if applicable for this file
        try:
            df = pd.read_csv("georef-italy-comune.csv", sep=';')
        except Exception as e_semicolon:
            print(f"Error reading with semicolon delimiter: {e_semicolon}")
            print("Could not read the CSV file with common delimiters or python engine.")
            df = None # Assign None if all attempts fail

if df is not None:
    print("Successfully loaded the CSV file.")
    display(df.head())

Error reading with comma delimiter: Error tokenizing data. C error: Expected 830 fields in line 35, saw 924

Error reading with python engine: Expected 830 fields in line 35, saw 924
Successfully loaded the CSV file.


,Geo Point,Geo Shape,Year,Official Code Ripartizione geografica,Official Name Ripartizione geografica,Official Code Regione,Official Name Regione,Official Code Provincia/Città metropolitana,Official Name Provincia/Città metropolitana,Official Code Comune,Official Name Comune,Official Name Comune upper case,Official Name Comune lower case,Iso 3166-3 Area Code,Type,Alternative name Comune,Capoluogo status
0,"37.73026869766225, 13.216545081380048","{""coordinates"": [[[[13.273804753107155, 37.723...",2022,5,Isole,19,Sicilia,282,Palermo,82010,Bisacquino,BISACQUINO,bisacquino,ITA,Comune,NaN,No
1,"39.26280802751059, 9.05340949610849","{""coordinates"": [[[9.07524236739022, 39.271186...",2022,5,Isole,20,Sardegna,292,Cagliari,92108,Elmas,ELMAS,elmas,ITA,Comune,NaN,No
2,"40.43013067291798, 9.165559262788904","{""coordinates"": [[[9.161360718306119, 40.36854...",2022,5,Isole,20,Sardegna,90,Sassari,90008,Benetutti,BENETUTTI,benetutti,ITA,Comune,NaN,No
3,"40.03108318688678, 9.497758258403227","{""coordinates"": [[[9.590789495347195, 40.03304...",2022,5,Isole,20,Sardegna,91,Nuoro,91088,Talana,TALANA,talana,ITA,Comune,NaN,No
4,"38.218008434729725, 15.668545248194734","{""coordinates"": [[[15.679229372029788, 38.2080...",2022,4,Sud,18,Calabria,280,Reggio di Calabria,80018,Campo Calabro,CAMPO CALABRO,campo calabro,ITA,Comune,NaN,No


In [ ]:
# Clean column names by removing leading/trailing whitespace
df.columns = df.columns.str.strip()

df = df.drop(columns=['Geo Shape',
                      'Year',
                      'Official Name Ripartizione geografica',
                      'Official Code Regione',
                      'Official Code Provincia/Città metropolitana',
                      'Official Code Comune',
                      'Official Name Comune upper case',
                      'Official Name Comune lower case',
                      'Iso 3166-3 Area Code',
                      'Type',
                      'Alternative name Comune',
                      'Capoluogo status'])

In [ ]:
df

,Geo Point,Official Code Ripartizione geografica,Official Name Regione,Official Name Provincia/Città metropolitana,Official Name Comune
0,"37.73026869766225, 13.216545081380048",5,Sicilia,Palermo,Bisacquino
1,"39.26280802751059, 9.05340949610849",5,Sardegna,Cagliari,Elmas
2,"40.43013067291798, 9.165559262788904",5,Sardegna,Sassari,Benetutti
3,"40.03108318688678, 9.497758258403227",5,Sardegna,Nuoro,Talana
4,"38.218008434729725, 15.668545248194734",4,Calabria,Reggio di Calabria,Campo Calabro
...,...,...,...,...,...
7899,"40.36068643408806, 15.741261655109243",4,Basilicata,Potenza,Paterno
7900,"44.140867573618735, 9.6987642938379",1,Liguria,La Spezia,Vernazza
7901,"43.058799488802556, 11.604423411779331",3,Toscana,Siena,San Quirico d'Orcia
7902,"45.318126637513025, 7.691680640642676",1,Piemonte,Torino,Oglianico


In [ ]:
from google.colab import files

# Esporta il DataFrame in un file CSV
df.to_csv('georef-italy-comune.csv', index=False, encoding='utf-8', sep=',')

# Scarica il file CSV
files.download('georef-italy-comune.csv')

print("File 'georef-italy-comune.csv' esportato e scaricato.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File 'georef-italy-comune.csv' esportato e scaricato.


In [ ]:
fact_startup_registry

,RegionKey,SectorKey,Data_Iscrizione_StartUp,Data_Inizio_Attivita,Data_Registro_Imprese,Denominazione,nome,Provincia,Comune,Regione,AtecoCode,Settore,AltoValoreTecnologico,SpeseRnD,ForzaLavoroTitoli,PossessoBrevetti,PrevalenzaFemminile,PrevalenzaGiovanile,PrevalenzaStraniera,Status,TipoSocieta,Year,valore di produzione,addetti,valore di capitale
0,10,9,25/01/2023,12/01/2023,25/01/2023,plw 1 societa' agricola s.r.l.,Plw 1,BG,BERGAMO,LOMBARDIA,72102,SERVIZI,SI,SI,SI,no data,no data,NO,NO,attiva,societa agricola,2023,NaN,NaN,Da 5 a 10mila euro
1,4,9,11/07/2017,12/04/2017,31/05/2017,gate 42 s.r.l. ...,Gate 42,NaN,TORRE DEL GRECO,CAMPANIA,621000,SERVIZI,NO,SI,no data,no data,NO,NO,NO,attiva,startup costituita a norma dell art 4 comma 10...,2017,100.001 - 500.000 euro,0-4,Da 100 a 250mila euro
2,8,9,23/12/2020,18/12/2020,02/02/2024,block-data 52 - societa' a responsabilita' lim...,Block-Data 52,RM,ROMA,LAZIO,621000,SERVIZI,NO,no data,no data,SI,NO,NO,NO,attiva,startup,2020,100.001 - 500.000 euro,NaN,Da 10 a 50mila euro
3,8,0,10/02/2022,10/01/2021,31/03/2025,e-solutions 78 - societa' a responsabilita' li...,E-Solutions 78,RM,ROMA,LAZIO,NC,NaN,NO,no data,SI,no data,NO,NO,NO,attiva,startup,2022,100.001 - 500.000 euro,NaN,Da 10 a 50mila euro
4,3,9,22/06/2023,08/06/2023,22/06/2023,before 1783 s.b.s.r.l.,Before 1783,RC,TERRANOVA SAPPO MINULIO,CALABRIA,621000,SERVIZI,NO,SI,no data,no data,Forte,NO,NO,attiva,societa benefit,2023,NaN,0-4,Da 5 a 10mila euro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12287,8,9,19/07/2023,23/11/2022,05/08/2022,brandsbrokers s.r.l.,Brandsbrokers,RM,ROMA,LAZIO,621000,SERVIZI,NO,no data,SI,no data,NO,NO,NO,attiva,societa benefit,2023,500.001 - 1.000.000 euro,0-4,Da 5 a 10mila euro
12288,18,9,13/01/2022,31/12/2021,14/12/2021,"404 societa' cooperativa sociale, in breve 40...",404,BZ,BOLZANO,TRENTINO-ALTO ADIGE,621000,SERVIZI,NO,SI,no data,no data,NO,NO,NO,attiva,societa cooperativa sociale,2022,0-100.000 euro,5-9,NaN
12289,11,1,12/01/2021,24/12/2020,12/01/2021,vallesina bio s.b. agricola srl,Vallesina,AN,MONSANO,MARCHE,011100,AGRICOLTURA/PESCA,NO,no data,SI,no data,NO,NO,NO,attiva,societa benefit agricola,2021,0-100.000 euro,NaN,Da 5 a 10mila euro
12290,10,9,24/03/2021,12/03/2021,24/03/2021,zero impack s.r.l. societa' benefit o in forma...,Zero,MI,MILANO,LOMBARDIA,621000,SERVIZI,NO,SI,no data,no data,Maggioritaria,NO,NO,attiva,societa benefit,2021,100.001 - 500.000 euro,0-4,Da 10 a 50mila euro


In [ ]:


# Mappatura completa
provincia_regione = {
    # LOMBARDIA
    'MI': 'LOMBARDIA', 'BG': 'LOMBARDIA', 'BS': 'LOMBARDIA', 'CO': 'LOMBARDIA',
    'CR': 'LOMBARDIA', 'LC': 'LOMBARDIA', 'LO': 'LOMBARDIA', 'MN': 'LOMBARDIA',
    'MB': 'LOMBARDIA', 'PV': 'LOMBARDIA', 'SO': 'LOMBARDIA', 'VA': 'LOMBARDIA',

    # LAZIO
    'RM': 'LAZIO', 'FR': 'LAZIO', 'LT': 'LAZIO', 'RI': 'LAZIO', 'VT': 'LAZIO',

    # CAMPANIA
    'NA': 'CAMPANIA', 'AV': 'CAMPANIA', 'BN': 'CAMPANIA', 'CE': 'CAMPANIA', 'SA': 'CAMPANIA',

    # VENETO
    'VE': 'VENETO', 'BL': 'VENETO', 'PD': 'VENETO', 'RO': 'VENETO', 'TV': 'VENETO', 'VR': 'VENETO', 'VI': 'VENETO',

    # EMILIA-ROMAGNA
    'BO': 'EMILIA-ROMAGNA', 'FC': 'EMILIA-ROMAGNA', 'FE': 'EMILIA-ROMAGNA', 'MO': 'EMILIA-ROMAGNA',
    'PR': 'EMILIA-ROMAGNA', 'PC': 'EMILIA-ROMAGNA', 'RA': 'EMILIA-ROMAGNA', 'RE': 'EMILIA-ROMAGNA', 'RN': 'EMILIA-ROMAGNA', 'FO' : 'EMILIA-ROMAGNA',

    # PIEMONTE
    'TO': 'PIEMONTE', 'AL': 'PIEMONTE', 'AT': 'PIEMONTE', 'BI': 'PIEMONTE', 'CN': 'PIEMONTE', 'NO': 'PIEMONTE', 'VB': 'PIEMONTE', 'VC': 'PIEMONTE',

    # TOSCANA
    'FI': 'TOSCANA', 'AR': 'TOSCANA', 'GR': 'TOSCANA', 'LI': 'TOSCANA', 'LU': 'TOSCANA', 'MS': 'TOSCANA', 'PI': 'TOSCANA', 'PO': 'TOSCANA', 'PT': 'TOSCANA', 'SI': 'TOSCANA',

    # SICILIA
    'PA': 'SICILIA', 'AG': 'SICILIA', 'CL': 'SICILIA', 'CT': 'SICILIA', 'EN': 'SICILIA', 'ME': 'SICILIA', 'RG': 'SICILIA', 'SR': 'SICILIA', 'TP': 'SICILIA',

    # PUGLIA
    'BA': 'PUGLIA', 'BT': 'PUGLIA', 'BR': 'PUGLIA', 'FG': 'PUGLIA', 'LE': 'PUGLIA', 'TA': 'PUGLIA',

    # LIGURIA
    'GE': 'LIGURIA', 'IM': 'LIGURIA', 'LA': 'LIGURIA', 'SV': 'LIGURIA', 'SP': 'LIGURIA',

    # MARCHE
    'AN': 'MARCHE', 'AP': 'MARCHE', 'FM': 'MARCHE', 'MC': 'MARCHE', 'PU': 'MARCHE', 'PS' : 'MARCHE',

    # ABRUZZO
    'AQ': 'ABRUZZO', 'CH': 'ABRUZZO', 'PE': 'ABRUZZO', 'TE': 'ABRUZZO',

    # UMBRIA
    'PG': 'UMBRIA', 'TR': 'UMBRIA',

    # CALABRIA
    'CS': 'CALABRIA', 'CZ': 'CALABRIA', 'KR': 'CALABRIA', 'RC': 'CALABRIA', 'VV': 'CALABRIA',

    # SARDEGNA
    'CA': 'SARDEGNA', 'CI': 'SARDEGNA', 'NU': 'SARDEGNA', 'OG': 'SARDEGNA', 'OR': 'SARDEGNA', 'OT': 'SARDEGNA', 'SS': 'SARDEGNA', 'SU': 'SARDEGNA',

    # FRIULI-VENEZIA GIULIA
    'UD': 'FRIULI-VENEZIA GIULIA', 'GO': 'FRIULI-VENEZIA GIULIA', 'PN': 'FRIULI-VENEZIA GIULIA', 'TS': 'FRIULI-VENEZIA GIULIA',

    # TRENTINO-ALTO ADIGE
    'BZ': 'TRENTINO-ALTO ADIGE', 'TN': 'TRENTINO-ALTO ADIGE',

    # MOLISE
    'CB': 'MOLISE', 'IS': 'MOLISE',

    # BASILICATA
    'PZ': 'BASILICATA', 'MT': 'BASILICATA',

    # VALLE D'AOSTA
    'AO': "VALLE D'AOSTA",

    # MARCHE
    'PS': "PESARO E URBINO",

    # EMILIA-ROMAGNA
    'FO': "FORLI-CESENA",

    # LIGURIA
    'SP': "LA SPEZIA"
}



In [ ]:
fact_startup_registry['Provincia'] = fact_startup_registry['Provincia'].astype(str).str.strip().str.upper()

fact_startup_registry['Regione_Mappata'] = fact_startup_registry['Provincia'].map(provincia_regione)

# Verifica
print(" Mappatura completata. Prime righe con Regione:")
display(fact_startup_registry[['Provincia', 'Regione_Mappata']].head())

 Mappatura completata. Prime righe con Regione:


,Provincia,Regione_Mappata
0,BG,LOMBARDIA
1,NAN,NaN
2,RM,LAZIO
3,RM,LAZIO
4,RC,CALABRIA


In [ ]:
from google.colab import files

# Esporta il DataFrame in un file CSV
fact_startup_registry.to_csv('fact_startup_registry.csv', index=False, encoding='utf-8', sep=',')

# Scarica il file CSV
files.download('fact_startup_registry.csv')

print("File 'fact_startup_registry.csv' esportato e scaricato.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File 'fact_startup_registry.csv' esportato e scaricato.
